In [1]:
import time

ROW = "ABCDEFGHI"
COL = "123456789"

In [2]:
def print_board(board):
    """Helper function to print board in a square."""
    print("-----------------")
    for i in ROW:
        row = ''
        for j in COL:
            row += (str(board[i + j]) + " ")
        print(row)
        
def board_to_string(board):
    """Helper function to convert board dictionary to string for writing."""
    ordered_vals = []
    for r in ROW:
        for c in COL:
            ordered_vals.append(str(board[r + c]))
    return ''.join(ordered_vals)

def is_complete(board):
    for r in ROW:
        for c in COL:
            if board[r + c] == 0:
                return False
    return True

def setup_constrains(board):
    constrains = {}
    d = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    for r in ROW:
        for c in COL:
            if board[r + c] != 0:
                constrains[r + c] = [board[r + c]]
            else:
                constrains[r + c] = d
    return constrains

In [3]:
def sub_sudoku():
    """ helper function to set index for 9 sub 3x3 sudokus """
    
    d = {}
    for i in range(9):
        d[i + 1] = []

    k = 0
    while k < 3:
        l1 = []
        l2 = []
        l3 = []
        for i in range(3):
            for j in range(3):
                l1.append(ROW[i] + COL[j + k * 3])
                l2.append(ROW[i + 3] + COL[j + k * 3])
                l3.append(ROW[i + 6] + COL[j + k * 3])
                d[k + 1] = l1
                d[k + 4] = l2
                d[k + 7] = l3
        k = k + 1

    return d

def check_constrains(cell, board, constrains):
    idx = list(cell)
    invalid = []
    constrain_values = [1, 2, 3, 4, 5, 6, 7, 8, 9]

    # row constrains
    for i in range(1, 10):
        if board[idx[0] + str(i)] != 0:
            v = board[idx[0] + str(i)]
            if v not in invalid:
                invalid.append(v)
    # column constrains
    for r in ROW:
        if board[r + idx[1]] != 0:
            v = board[r + idx[1]]
            if v not in invalid:
                invalid.append(v)
    # 3x3 block constrains
    d = sub_sudoku()
    for k,v in d.items():
        if cell in v:
            for i in v:
                if board[i] !=0:
                    invalid.append(board[i])

    for v in invalid:
        if v in constrain_values:
            constrain_values.remove(v)
    # print(constrains)
    constrains[cell] = constrain_values

def select_unassigned_assignment(constrains):
    mrv = {}
    tmp = 10
    idx = ''
    values = []
    for k,v in constrains.items():
        if len(v) > 0:
            if len(v) < tmp:
                tmp = len(v)
                idx = k
                values = v
    mrv[idx] = values
    return mrv

def forward_checking(idx, constrains, board):
    del constrains[idx]
    for k, v in constrains.items():
        check_constrains(k, board, constrains)
        for idx, val in constrains.items():
            if len(val) == 0:
                return False
    return True

In [4]:
def bts(board):
    """Takes a board and returns solved board."""
    initial_constrains = setup_constrains(board)
    constrains = {}

    # update constrains
    for k, v in initial_constrains.items():
        if len(v) != 1:
            check_constrains(k, board, constrains)
    solved_board = backtracking(board, constrains)

    return solved_board

def backtracking(board, constrains):
    
    if is_complete(board):
        result = board
        return result
    
    d = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    mrv = select_unassigned_assignment(constrains)
    key = ''
    for k, v in mrv.items():
        key = k

    for i in d:
        old_constrains = dict(constrains)
        old_board = dict(board)

        if i in constrains[key]:
            board[key] = i
            if forward_checking(key, constrains, board) != 0:
                result = backtracking(board, constrains)
                if result != None:
                    return result

        for k, v in old_constrains.items():
            constrains[k] = old_constrains[k]
        for k, v in old_board.items():
            board[k] = old_board[k]
        board[key] = 0

    return None

In [5]:
# Here is a list of sudoku boards to test the program

sudoku_list = \
["003020600900305001001806400008102900700000008006708200002609500800203009005010300",
"000260701680070090190004500820100040004602900050003028009300074040050036703018000",
"000100702030950000001002003590000301020000070703000098800200100000085060605009000",
"094000130000000000000076002080010000032000000000200060000050400000008007006304008",
"000000000000942080160000029000000008906000001400250000004000000020008090050000700",
"000007000090001000000045006000020000036000410500000809000000004000018000081500032",
"052470000060000000000008010400000009700950000020040030000800090000003706000091000",
"090000000001006000060080070300000010000039000000050002170400028000003000086000057",
"000005000020004010030080020000008400800600000090010705006000000950003060003000001",
"500068000000000060042050000000800900001000040903000620700001009004200003080000000"]


In [6]:
if __name__ == '__main__':
    
    #Please feel free to change the index of sudoku_list to test other sudoku   
    line = sudoku_list[0]
    
    # Parse boards to dict representation, scanning board L to R, Up to Down
    board = { ROW[r] + COL[c]: int(line[9*r+c])
              for r in range(9) for c in range(9)}

    print("Sudoku to be solved: ")
    print_board(board)

    start = time.time()
    solved_board = bts(board)
    runtime = time.time() - start
    
    print("\nSudoku result: ")
    print_board(solved_board)

    print("\nRuntime: ", runtime)


Sudoku to be solved: 
-----------------
0 0 3 0 2 0 6 0 0 
9 0 0 3 0 5 0 0 1 
0 0 1 8 0 6 4 0 0 
0 0 8 1 0 2 9 0 0 
7 0 0 0 0 0 0 0 8 
0 0 6 7 0 8 2 0 0 
0 0 2 6 0 9 5 0 0 
8 0 0 2 0 3 0 0 9 
0 0 5 0 1 0 3 0 0 

Sudoku result: 
-----------------
4 8 3 9 2 1 6 5 7 
9 6 7 3 4 5 8 2 1 
2 5 1 8 7 6 4 9 3 
5 4 8 1 3 2 9 7 6 
7 2 9 5 6 4 1 3 8 
1 3 6 7 9 8 2 4 5 
3 7 2 6 8 9 5 1 4 
8 1 4 2 5 3 7 6 9 
6 9 5 4 1 7 3 8 2 

Runtime:  0.11265277862548828


In [7]:
# Here is the list of answers to the previous sudoku boards 
# for you to check the answers!

sudoku_result = \
["483921657967345821251876493548132976729564138136798245372689514814253769695417382",
"435269781682571493197834562826195347374682915951743628519326874248957136763418259",
"956138742237954816481672953594867321128593674763421598879246135312785469645319287",
"794582136268931745315476982689715324432869571157243869821657493943128657576394218",
"249186573735942186168375429512697348976834251483251967694723815327518694851469732",
"653287941794631258128945376819724563236859417547163829965372184372418695481596732",
"152479683368215974974638512416387259783952461529146837237864195891523746645791328",
"894317265731526894562984173358642719247139586619758432173465928925873641486291357",
"168295374529734618437186529312578496875649132694312785786951243951423867243867951",
"597468132318927564642153897456832971821796345973514628735641289164289753289375416"
]
